In [ ]:
import pandas as pd
import numpy as np
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
import plotly.express as px
# コールバックに必要なライブラリのインポート
from dash.dependencies import Input, Output, State

## データの読み込み
* df_pref : 48都道府県のTOP50店をランキング形式でデータフレーム化.  df_kindのジャンルレコードと店名（nameレコード）が一致した場合、順位を新しいカラムとして追加する
* df_pref_of_parking : 都道府県ごとに駐車場の有無についてカウントしたデータフレーム
* df_kind : rankレコードを追加して、順位をレコードとして扱えるようにする


In [ ]:
df_pref = pd.read_csv('../data/cleaning_pref.csv')
display(df_pref.head(3))

df_pref_of_parking = pd.read_csv('../data/groupby_parking.csv')
display(df_pref_of_parking.head())

df_kind = pd.read_csv('../data/cleaning_kind_of_rank.csv')
display(df_kind.head(3))

df_kind_of_pref = pd.read_csv('../data/groupby_kind.csv')
display(df_kind_of_pref.head(3))

## 種類ごとの分類でランクインした店舗に対してフラグを追加.  
旧型の種類ごとのランキングのデータフレームに対して用いる

In [ ]:
# for i, col in enumerate(df_kind.columns):
#   # df_kindのrank以外のレコードを条件にマージする
#   if i == 1:
#     df_kind_rank = pd.merge(df_pref, df_kind[['rank', col]], left_on='name', right_on=col, how='outer', suffixes=['','_'+col])
#     df_kind_rank.drop(col, axis=1, inplace=True)
#   elif i > 1:
#     df_kind_rank = pd.merge(df_kind_rank, df_kind[['rank', col]], left_on='name', right_on=col, how='outer', suffixes=['','_'+col])
#     df_kind_rank.drop(col, axis=1, inplace=True)

# df_kind_rank = df_kind_rank.rename(columns={'rank':'rank_ラーメン'})
# df_kind_rank[['name', 'point','rank_ラーメン', 'rank_つけ麺', 'rank_汁なし', 'rank_冷やし中華', 'rank_醤油', 'rank_味噌', 'rank_塩', 'rank_豚骨',
#               'rank_豚骨醤油', 'rank_豚骨魚介', 'rank_鶏白湯', 'rank_煮干し', 'rank_担々麺','rank_カレー', 'rank_塩豚骨']].query('`rank_ラーメン` <= 10').sort_values(by = 'rank_ラーメン')

# df_kind_rank.loc[~df_kind_rank['rank_味噌'].isnull()].sort_values('rank_味噌')

## スタイルシートの作成.  

In [ ]:
# bootstrapスタイルシートの読み込み
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

## checkbox要素の準備

In [ ]:
# optionに設定される要素の準備
vars_parking = [park for park in np.unique(df_pref['parking'].values)]
week_dict = {
  '月':'mon_holiday',
  '火':'tue_holiday',
  '水':'wed_holiday',
  '木':'thu_holiday',
  '金':'fri_holiday',
  '土':'sat_holiday',
  '日':'sun_holiday'
}
vars_pref = ['']
vars_pref = np.append(vars_pref,[pref for pref in np.unique(df_pref_of_parking['pref'].values)])

vars_kind = [kind for kind in np.unique(df_kind['kind'].values)]


## サイドバーのコーディング

In [ ]:
# サイドバーのレイアウト定義
sidebar = html.Div(
  [
    # タイトルの表示
    dbc.Row(
      [
        html.H5('RamenDB Plotly', style={'margin-top': '12px', 'margin-left': '24px'})
      ],
      style={"height": "5vh"},
      className='bg-primary text-white font-italic'
    ),
    # 1枚目のマップに対応したスライダー部分
    dbc.Row(
      [
        # インプット用のコンポーネント
        html.Div(
          [
            # 駐車場の有無,空いている店舗を選択できる
            html.H3('駐車場の有無', style={'margin-top': '8px', 'margin-bottom': '4px'}, className='font-weight-bold'),
            dcc.Checklist(
              id='check_parking',
              # 表示する要素辞書形式で指定
              options=[{'label': x, 'value': x} for x in vars_parking], 
              # 初期で指定する要素の指定
              value=['駐車場あり', '駐車場なし', '情報なし'],
              # 要素の縦表示か横表示かの指定
              inline=False,
              style={'width': '320px'}
            ),
            html.H3('定休日の選択', style={'margin-top': '16px', 'margin-bottom': '4px'}, className='font-weight-bold'),
            dcc.Checklist(
              id='check_holiday',
              options=[{'label': key, 'value': value} for key, value in week_dict.items()], 
              value=[], 
              inline=True,
              style={'width': '120px'}
            ),
            # 都道府県のドロップダウンを表示
            html.P('調べたい都道府県を選んでください', style={'margin-top': '8px', 'margin-bottom': '4px'}, className='font-weight-bold'),
            dcc.Dropdown(
              id='select_pref',
              # 表示する要素辞書形式で指定
              options=[{'label': value, 'value': value} for value in vars_pref], 
              # 初期で指定する要素の指定
              value='',
              style={'width': '120px'}
            ),
            html.Button(id='apply_button',
                        n_clicks=0,
                        children='apply',
                        style={'margin-top': '16px'},
                        className='bg-dark text-white'
            ),
            html.Hr(),
            # 種類ごとにTOP50の名店のみ表示されるボタンを表示
            html.P('調べたいジャンルを選んでください', style={'margin-top': '8px', 'margin-bottom': '4px'}, className='font-weight-bold'),
            dcc.Dropdown(
              id='selected_kind',
              # 表示する要素辞書形式で指定
              options=[{'label': value, 'value': value} for value in vars_kind], 
              # 初期で指定する要素の指定
              value='ラーメン',
              style={'width': '120px'}
            ),
            html.Button(id='apply_button_rank',
                        n_clicks=0,
                        children='apply_rank',
                        style={'margin-top': '16px'},
                        className='bg-dark text-white'
            ),
            html.Hr()
          ]
        )
      ],
      style={'height': '50vh', 'margin': '8px'}
    )
  ]
)

## コンテンツ箇所のマップの描画と店舗数の定義(初期に使用される)

一列目画像

In [ ]:
# hover時に表示するレコードを指定
cols = df_pref.columns.drop([
                              'address', 
                              'latitude', 
                              'longitude', 
                              'name',
                              'mon_holiday',
                              'tue_holiday',
                              'wed_holiday',
                              'thu_holiday',
                              'fri_holiday',
                              'sat_holiday',
                              'sun_holiday',
                              'pref'
                             ])

# scatter_mapboxグラフの作成
fig = px.scatter_mapbox(
  # データフレームと、緯度、軽度の指定
  df_pref, 
  lat="latitude", 
  lon="longitude", 
  # ホバー時の表示設定
  hover_name="name",
  hover_data=cols,
  color_discrete_sequence=["#636EFA"], 
  opacity=0.8,
  #詳細設定
  zoom=3, 
  height=450,
  width=450
)
# mapの種類を指定
fig.update_layout(mapbox_style="carto-positron")
# 余白の調整
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

store_num = df_pref.shape[0]

# 選択した都道府県の駐車場情報の比率を円グラフで可視化
df_pref_of_parking_new = df_pref_of_parking.groupby('parking').sum().reset_index()
df_pref_of_parking_new['pref'] = '全国'

#対象都道府県の宣言文
pref_park =  '全国の駐車場の有無割合'

fig_pie = px.pie(
  df_pref_of_parking_new,
  values='count', 
  names='parking'
)

# 余白の調整
fig_pie.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})



2枚目の画像

In [ ]:
# hover時に表示するレコードを指定
cols_rank = df_kind.columns.drop([
                              'kind', 
                              'latitude', 
                              'longitude',
                              'name', 
                              'mon_holiday',
                              'tue_holiday',
                              'wed_holiday',
                              'thu_holiday',
                              'fri_holiday',
                              'sat_holiday',
                              'sun_holiday',
                            ])

# scatter_mapboxグラフの作成
fig_rank = px.scatter_mapbox(
  # データフレームと、緯度、軽度の指定
  df_kind.loc[df_kind['kind'] == 'ラーメン'], 
  lat="latitude", 
  lon="longitude", 
  # ホバー時の表示設定
  hover_name="name",
  # name要素以外に表示したいデータの追加
  hover_data=cols_rank,
  color_discrete_sequence=["#6363FA"], 
  opacity=0.8,
  #詳細設定
  zoom=3, 
  height=450,
  width=450
)
# mapの種類を指定
fig_rank.update_layout(mapbox_style="carto-positron")
# 余白の調整
fig_rank.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

#対象都道府県の宣言文
kind_prefs =  '都道府県の割合'

fig_pie_pref = px.pie(
  df_kind_of_pref.loc[df_kind_of_pref['kind'] == 'ラーメン'], 
  values='count', 
  names='pref'
)

## コールバックの設定

一列目のマップのコールバック関数

In [ ]:
@app.callback(
  Output(component_id='ramen_map', component_property='figure'),
  Output(component_id='store_number', component_property='children'),
  Output(component_id='pref_parking', component_property='children'),
  Output(component_id='parking_pie', component_property='figure'),
  Input(component_id='apply_button', component_property='n_clicks'),
  State(component_id='check_parking', component_property='value'),
  State(component_id='check_holiday', component_property='value'),
  State(component_id='select_pref', component_property='value')
)
def update_map(n_clicks, selected_parking, selected_holidays, selected_pref):
  # 新しいデータフレームの枠を作成する
  df_pref_new = pd.DataFrame(columns=df_pref.columns)
  #selected_parking, selected_holidaysはリスト形式で返却される
  #駐車場に関して指定した要素で、データフレームの絞り込みを行う
  # パーキング要素が指定されていれば発動
  if len(selected_parking):
    for value in selected_parking:
      df_pref_new = pd.concat([df_pref_new, df_pref[df_pref['parking'] == value]])
  
  if len(selected_holidays):
    for col in selected_holidays:
      # 定休日でない店舗を表示する
      # 渡されたカラムが0であるインデックスのみ表示
      df_pref_new = df_pref_new[df_pref_new[col] == 0]

  if len(selected_pref):
    df_pref_new = df_pref_new.query(f"pref == '{selected_pref}'")
    # 駐車場比率に関するデータフレームも整形
    df_pref_of_parking_new = df_pref_of_parking.query(f"pref == '{selected_pref}'")
  else :
    selected_pref = '全国'
    # 全都道府県の駐車場比率を求められるようフレームを再集計する
    df_pref_of_parking_new = df_pref_of_parking.groupby('parking').sum().reset_index()
    df_pref_of_parking_new['pref'] = '全国'
  
  # 上記の処理によりデータフレーム完成
  # scatter_mapboxグラフの作成
  fig = px.scatter_mapbox(
    # データフレームと、緯度、軽度の指定
    df_pref_new, 
    lat="latitude", 
    lon="longitude", 
    # ホバー時の表示設定
    hover_name="name",
    hover_data=cols,
    color_discrete_sequence=["#636EFA"], 
    opacity=1,
    #詳細設定
    zoom=7, 
    height=450,
    width=450
  )
  # mapの種類を指定
  fig.update_layout(mapbox_style="carto-positron")
  # 余白の調整
  fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

  #対象店舗数のカウント
  store_num = df_pref_new.shape[0]
  store_num = '対象店舗数：' + str(store_num)

  #対象都道府県の宣言文
  pref_park = selected_pref + 'の駐車場の有無割合'

  # 選択した都道府県の駐車場情報の比率を円グラフで可視化
  fig_pie = px.pie(
    df_pref_of_parking_new,
    values='count', 
    names='parking'
  )
  
  return fig, store_num, pref_park, fig_pie

2枚目の画像のコールバック関数

In [ ]:
@app.callback(
  Output(component_id='rank_map', component_property='figure'),
  Output(component_id='rank_of_pref', component_property='figure'),
  Input(component_id='apply_button_rank', component_property='n_clicks'),
  State(component_id='selected_kind', component_property='value')
)
def update_map(n_clicks, selected_kind):
  # データフレームの作成を行う
  # 選択された種類の要素でデータフレームの絞り込みを行う
  df_kind_of_rank = df_kind.loc[df_kind['kind'] == selected_kind].reset_index().rename(columns={'index':'rank'})
  # 新しく作成したランクカラムを順位になるよう1を足して補正する
  df_kind_of_rank['rank'] = df_kind_of_rank['rank'].apply(lambda x : int(x)+1)
  # 選択された種類の要素で絞り込み
  df_kind_of_pref_new = df_kind_of_pref.loc[df_kind_of_pref['kind'] == selected_kind]
  
  # 上記の処理によりデータフレーム完成
  # scatter_mapboxグラフの作成
  fig_rank = px.scatter_mapbox(
    # データフレームと、緯度、軽度の指定
    data_frame = df_kind_of_rank, 
    lat="latitude", 
    lon="longitude", 
    # ホバー時の表示設定
    hover_name="name",
    hover_data=cols,
    color_discrete_sequence=["#636EFA"], 
    opacity=1,
    #詳細設定
    zoom=7, 
    height=450,
    width=450
  )
  # mapの種類を指定
  fig_rank.update_layout(mapbox_style="carto-positron")
  # 余白の調整
  fig_rank.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

  fig_pie_pref = px.pie(
    df_kind_of_pref_new,
    values='count', 
    names='pref'
  )

  return fig_rank, fig_pie_pref

## コンテンツ箇所の画像の貼り付け

In [ ]:
content_1 = html.Div(
  [
    dbc.Row(
      [
        html.H3('条件に従うラーメン店一覧'),
        # dcc.graphコンポーネントを利用して、グラフをプロットする
        dcc.Graph(id='ramen_map', figure=fig),
        html.P(id='store_number', children=[store_num]),
        html.P(id='pref_parking', children=[pref_park]),
        dcc.Graph(id='parking_pie', figure=fig_pie)
      ]
    )
  ]
)
content_2 = html.Div(
  [
    dbc.Row(
      [
        html.H3(' 種類ごとのTOP100の名店'),
        # dcc.graphコンポーネントを利用して、グラフをプロットする
        dcc.Graph(id='rank_map', figure=fig_rank),
        dcc.Graph(id='rank_of_pref', figure=fig_pie_pref)
      ]
    )
  ]
)

## アプリの起動

In [ ]:
# 画面全体のgridの縦比を、1：3に分割する
app.layout = dbc.Container(
  [
    dbc.Row(
      [
        dbc.Col(sidebar, width=2, className='bg-light'),
        dbc.Col(content_1, width=5),
        dbc.Col(content_2, width=5)
      ],
      style={"height": "200vh"}
    ),
  ],
  # 不要な余白の除去
  fluid=True
)

if __name__ == "__main__":
  app.run_server(debug=True, port=1234)


In [ ]:
df_kind[['kind', 'pref']].groupby(['kind', 'pref']).count().reset_index()

In [ ]:
np.unique(df_pref_of_parking['pref'].values)

In [ ]:
df_pref_of_parking_new = df_pref_of_parking.groupby('parking').sum().reset_index()
df_pref_of_parking_new['pref'] = '全国'
df_pref_of_parking_new